In [68]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [69]:
import pandas as pd
import technical_analysis.backtest_kz as bt
from data_pipelines.data_manipulation import DataManipulation
import warnings
import numpy as np
import pandas_ta as ta
warnings.filterwarnings('ignore')
warnings.simplefilter(action = 'ignore', category = pd.errors.PerformanceWarning)
%matplotlib inline

In [70]:
SYMBOL = 'BTC-USD'
scale = 1
range = [5, 10, 15, 20, 25, 30, 50, 99]
range = [i*scale for i in range]
period = '4y'
interval = '1d'
start_date = ''
end_data = ''
source = 'yahoo'

In [71]:
data = DataManipulation(SYMBOL, source, range, period=period, interval=interval, scale=scale, prefix_path='..')

In [72]:
data.df.tail()

,Open,High,Low,Close,Volume,sma_5,sma_10,sma_15,sma_20,sma_25,...,stoch_d,ich_kline,ich_tline,FISHERT_9_1,FISHERTs_9_1,log_rt,candlestick_pattern,candle_label,d_r,feature_label
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-11-04,20208.7695,21209.5605,20188.0195,21147.2305,64072727950,20499.5539,20560.3479,20199.5049,19966.9559,19807.3670,...,37.7325,19764.6914,20647.8145,0.7621,0.9563,0.0453,CDLBELTHOLD_Bull,1.0000,0.0464,1.0000
2022-11-05,21144.8320,21446.8867,21097.6348,21282.6914,37846047609,20656.9375,20611.5729,20340.1865,20067.6857,19896.6180,...,71.0659,19883.3545,20766.4775,0.9510,0.7621,0.0064,NO_PATTERN,0.0000,0.0064,1.0000
2022-11-06,21285.0566,21345.3770,20920.1914,20926.4863,35082693210,20745.1801,20675.6379,20454.7396,20136.4722,19967.3796,...,66.6667,19883.3545,20766.5635,1.1479,0.9510,-0.0169,CDLCLOSINGMARUBOZU_Bear,-1.0000,-0.0167,1.0000
2022-11-07,20924.6211,21053.2461,20489.9727,20602.8164,53510852236,20833.8426,20676.3844,20523.7935,20199.8922,20016.1761,...,33.3333,19883.3545,20766.5635,1.0032,1.1479,-0.0156,CDL3OUTSIDE_Bear,-1.0000,-0.0155,0.0000
2022-11-08,20620.4492,20662.3379,19409.4863,19476.9219,70345875456,20687.2293,20542.2289,20532.5501,20216.7615,20027.8267,...,-0.0000,20108.9287,20428.1865,0.4427,1.0032,-0.0562,CDLBELTHOLD_Bear,-1.0000,-0.0546,0.0000


In [73]:
df = data.df
sample  = df[['Open','High','Low','Close','Volume']].copy()
sample.drop(columns=['Open','High','Low'], axis=1, inplace=True)
sample['st_hisse'] = np.nan
sample['st_stoch'] = np.nan
sample['st_fisher'] = np.nan
sample['st_ich'] = np.nan
sample['st_mfi'] = np.nan

In [74]:
for index, datetime in enumerate(df.index):
        current_datetime = datetime
        ema5 = df['ema_5'].iloc[index]
        sma10 = df['sma_10'].iloc[index]
        macd = df['macd'].iloc[index]
        macds = df['macdsignal'].iloc[index]
        ichkline = df['ich_kline'].iloc[index]
        ichtline = df['ich_tline'].iloc[index]
        close = df['Close'].iloc[index]
        dmu = df['dmi_up_15'].iloc[index]
        dmd = df['dmi_down_15'].iloc[index]
        stk = df['stoch_k'].iloc[index]
        std = df['stoch_d'].iloc[index]
        fischer = df['FISHERT_9_1'].iloc[index]
        mfi = df['mfi_20'].iloc[index]

        if ichtline >= ichkline:
                sample['st_ich'].iloc[index] = 1
        else:
                sample['st_ich'].iloc[index] = 0

        if stk >= std:
                sample['st_stoch'].iloc[index] = 1
        else:
                sample['st_stoch'].iloc[index] = 0

        if fischer >= 2.5:
                sample['st_fisher'].iloc[index] = 3
        elif fischer >= -2.5:
                sample['st_fisher'].iloc[index] = 2
        else:
                sample['st_fisher'].iloc[index] = 1 

        if mfi >= 75:
                sample['st_mfi'].iloc[index] = 3
        elif mfi >= 25:
                sample['st_mfi'].iloc[index] = 2
        else:
                sample['st_mfi'].iloc[index] = 1 

        pattern1 = ema5 >= sma10
        pattern2 = macd > macds
        pattern3 = ichkline < close
        pattern4 = dmu >= dmd
        pattern5 = sma10 < close
        pattern6 = stk > std

        all_pattern = (pattern1 and pattern2 and pattern3 and pattern4 and pattern5 and pattern6)
        if all_pattern:
                sample['st_hisse'].iloc[index] = 1
        else:
                sample['st_hisse'].iloc[index] = 0

In [75]:
def norm_features_ind(sdf, df2, ind, range, dividend):
    k = 0
    for i in range:
        sdf[f'st_{ind}_{i}'] = (dividend - df2[f'{ind}_{i}']) / dividend
        if k % 2 == 1:
            sdf[f'st_dt_{ind}_{range[k-1]}_{range[k]}'] = \
                    (df2[f'{ind}_{range[k-1]}'] - df2[f'{ind}_{range[k]}']) / dividend
        k += 1


In [76]:
norm_features_ind(sample, df, 'ema', range, df.Close)
norm_features_ind(sample, df, 'mfi', range, 100)
norm_features_ind(sample, df, 'sma', range, df.Close)
norm_features_ind(sample, df, 'tema', range, df.Close)
norm_features_ind(sample, df, 'kama', range, df.Close)
norm_features_ind(sample, df, 'rsi', range, 100)

In [77]:
sample['month'] = sample.index.month
sample['weekday'] = sample.index.weekday
sample['is_quarter_end'] = sample.index.is_quarter_end*1
sample['candle'] = df.candle_label
sample['vol_delta'] = sample['Volume'].pct_change()
sample['log_return'] = df.log_rt
sample.drop(columns=['Close', 'Volume'], axis=1, inplace=True)

In [78]:
sample.tail()

,st_hisse,st_stoch,st_fisher,st_ich,st_mfi,st_ema_5,st_ema_10,st_dt_ema_5_10,st_ema_15,st_ema_20,...,st_dt_rsi_25_30,st_rsi_50,st_rsi_99,st_dt_rsi_50_99,month,weekday,is_quarter_end,candle,vol_delta,log_return
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-11-04,1.0000,1.0000,2.0000,1.0000,2.0000,0.0259,0.0344,0.0085,0.0419,0.0476,...,0.0206,0.4985,0.5350,0.0365,11,4,0,1.0000,0.4822,0.0453
2022-11-05,1.0000,1.0000,2.0000,1.0000,2.0000,0.0214,0.0331,0.0117,0.0420,0.0485,...,0.0218,0.4949,0.5336,0.0387,11,5,0,0.0000,-0.4093,0.0064
2022-11-06,0.0000,0.0000,2.0000,1.0000,2.0000,0.0032,0.0136,0.0105,0.0225,0.0292,...,0.0158,0.5043,0.5368,0.0324,11,6,0,-1.0000,-0.0730,-0.0169
2022-11-07,0.0000,0.0000,2.0000,1.0000,2.0000,-0.0083,-0.0015,0.0068,0.0062,0.0127,...,0.0107,0.5128,0.5396,0.0269,11,0,0,-1.0000,0.5253,-0.0156
2022-11-08,0.0000,1.0000,2.0000,1.0000,2.0000,-0.0444,-0.0486,-0.0042,-0.0448,-0.0402,...,-0.0029,0.5405,0.5494,0.0089,11,1,0,-1.0000,0.3146,-0.0562
